In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

In [2]:
df = pd.read_csv("./insurance.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.3,random_state=25)

In [5]:
X_train_scaled = X_train.copy()
X_train_scaled['age']/=100
X_test_scaled = X_test.copy()
X_test_scaled['age']/=100

In [7]:
model = keras.Sequential(
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')
)
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.fit(X_train_scaled,y_train,epochs=5000)

Epoch 1/5000
1/1 [==============================] - 1s 1s/step - loss: 0.6482 - accuracy: 0.5789
Epoch 2/5000
1/1 [==============================] - 0s 20ms/step - loss: 0.6480 - accuracy: 0.5789
Epoch 3/5000
1/1 [==============================] - 0s 17ms/step - loss: 0.6477 - accuracy: 0.5789
Epoch 4/5000
1/1 [==============================] - 0s 18ms/step - loss: 0.6475 - accuracy: 0.5789
Epoch 5/5000
1/1 [==============================] - 0s 16ms/step - loss: 0.6472 - accuracy: 0.5789
Epoch 6/5000
1/1 [==============================] - 0s 19ms/step - loss: 0.6470 - accuracy: 0.5789
Epoch 7/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.6467 - accuracy: 0.5789
Epoch 8/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.6464 - accuracy: 0.5789
Epoch 9/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.6462 - accuracy: 0.5789
Epoch 10/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.6459 - accuracy: 0.5789
Epoch 11/5000


In [8]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 314ms/step - loss: 0.4275 - accuracy: 0.8889


[0.427470862865448, 0.8888888955116272]

In [9]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 176ms/step


array([[0.747135  ],
       [0.34992838],
       [0.22843245],
       [0.4919505 ],
       [0.7686708 ],
       [0.870502  ],
       [0.4050438 ],
       [0.2609498 ],
       [0.61906004]], dtype=float32)

In [12]:
coef,intercept = model.get_weights()
coef,intercept

(array([[5.871531 ],
        [1.0675523]], dtype=float32),
 array([-2.7437818], dtype=float32))

In [13]:
def sigmoid(x):
    import math
    return 1/(1+math.exp(-x))

In [14]:
sigmoid(10)

0.9999546021312976

In [15]:
def prediction_function(age,afford):
    weighted_sum = coef[0]*age + coef[1]*afford + intercept
    return sigmoid(weighted_sum)

In [16]:
prediction_function(0.47,1)

0.7471349726555601

In [17]:
prediction_function(0.18,1)

0.3499283861041912

## GD from scratch

In [18]:
def log_loss(y_true,y_predicted):
    epsilon = 1e-15
    y_pred_new = [ max(i,epsilon)  for i in y_predicted]
    y_pred_new = [ min(i,1-epsilon)  for i in y_pred_new]
    y_pred_new = np.array(y_pred_new)
    return - np.mean(
        y_true*np.log(y_pred_new) + (1-y_true)*np.log(1-y_pred_new)
    )

In [19]:
def sigmoid_numpy(X):
    return 1/(1+ np.exp(-X))

In [41]:
def gradient_descent(age,afford,y_true,epcohs,threshold):
    # w1 w2 bias
    w1 = w2 =1
    bias=0
    rate=0.1
    n = len(age)
    
    
    
    for i in range(epcohs):
        weighted_sum = (w1*age + w2*afford + bias)
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true,y_predicted)
        
        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted - y_true))
        w2d = (1/n)*np.dot(np.transpose(age),(y_predicted - y_true))
        biasd = np.mean(y_predicted - y_true)
        
        w1 = w1 - rate*w1d
        w2 = w2 - rate*w2d
        bias = bias - rate*biasd
        
        print(f"EPoch:{i}, w1:{w1},w2:{w2},bias:{bias},loss:{loss}")
        if loss<=threshold:
            break
    return w1,w2,bias  

In [42]:
gradient_descent( X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000,0.4433)

EPoch:0, w1:0.9977918068798303,w2:0.9977918068798303,bias:-0.015853823845437234,loss:0.6482464826346737
EPoch:1, w1:0.9957148108213941,w2:0.9957148108213941,bias:-0.03137837797933386,loss:0.645539624247212
EPoch:2, w1:0.9937667624496015,w2:0.9937667624496015,bias:-0.04657943368823358,loss:0.642955012638023
EPoch:3, w1:0.9919453767170439,w2:0.9919453767170439,bias:-0.06146283454258006,loss:0.6404875622620778
EPoch:4, w1:0.9902483383412549,w2:0.9902483383412549,bias:-0.07603448338360136,loss:0.6381323172146325
EPoch:5, w1:0.9886733070465037,w2:0.9886733070465037,bias:-0.09030032982998096,loss:0.6358844556505607
EPoch:6, w1:0.9872179225985552,w2:0.9872179225985552,bias:-0.10426635832703349,loss:0.6337392933372068
EPoch:7, w1:0.9858798096231419,w2:0.9858798096231419,bias:-0.11793857675571459,loss:0.6316922863907557
EPoch:8, w1:0.9846565822010618,w2:0.9846565822010618,bias:-0.1313230056137607,loss:0.6297390332480973
EPoch:9, w1:0.9835458482348467,w2:0.9835458482348467,bias:-0.14442566777659

(3.1364597335409234, 3.1364597335409234, -2.9883910751390963)

In [34]:
coef,intercept

(array([[5.871531 ],
        [1.0675523]], dtype=float32),
 array([-2.7437818], dtype=float32))

## Neural Network from scratch

In [65]:
class NN():
    def __init__(self):
        
        self.w1=1
        self.w2=1
        self.bias =0
    def sigmoid_numpy(self,X):
        return 1/(1+ np.exp(-X))

    def predict(self,x_test):
        weighted_sum = self.w1*x_test['age'] + self.w2*x_test['affordibility'] + self.bias
        return self.sigmoid_numpy(weighted_sum)
        
    def fit(self,x,y,epochs,loss_threshold):
        self.w1,self.w2,self.bias = self.gradient_descent(x['age'],x['affordibility'],y,epochs,loss_threshold)
    
    def gradient_descent(self,age,afford,y_true,epcohs,threshold):
        # w1 w2 bias
        w1 = w2 =1
        bias=0
        rate=0.1
        n = len(age)
        
        
        
        for i in range(epcohs):
            weighted_sum = (w1*age + w2*afford + bias)
            y_predicted = sigmoid_numpy(weighted_sum)
            loss = log_loss(y_true,y_predicted)
            
            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted - y_true))
            w2d = (1/n)*np.dot(np.transpose(age),(y_predicted - y_true))
            biasd = np.mean(y_predicted - y_true)
            
            w1 = w1 - rate*w1d
            w2 = w2 - rate*w2d
            bias = bias - rate*biasd
            
            if i%50==0:
                print(f"EPoch:{i}, w1:{w1},w2:{w2},bias:{bias},loss:{loss}")
            if loss<=threshold:
                break
        return w1,w2,bias  

In [66]:
customModel = NN()
customModel.fit(X_train_scaled,y_train,epochs=500,loss_threshold=0.4463)

EPoch:0, w1:0.9977918068798303,w2:0.9977918068798303,bias:-0.015853823845437234,loss:0.6482464826346737
EPoch:50, w1:1.0078495178302687,w2:1.0078495178302687,bias:-0.5060505345818135,loss:0.5932709212547838
EPoch:100, w1:1.120854075326724,w2:1.120854075326724,bias:-0.7306894938389791,loss:0.5816057104988627
EPoch:150, w1:1.2526360888198216,w2:1.2526360888198216,bias:-0.8945944432854235,loss:0.5742029156695689
EPoch:200, w1:1.3849107702250216,w2:1.3849107702250216,bias:-1.0427598288747804,loss:0.5684288549883295
EPoch:250, w1:1.514243005341211,w2:1.514243005341211,bias:-1.184464185510209,loss:0.5638918855488992
EPoch:300, w1:1.640145264468999,w2:1.640145264468999,bias:-1.321866193774574,loss:0.5604195004129212
EPoch:350, w1:1.7627104477343936,w2:1.7627104477343936,bias:-1.4556797152194296,loss:0.5578870087394532
EPoch:400, w1:1.882141230012626,w2:1.882141230012626,bias:-1.5863001981789164,loss:0.5561898689297727
EPoch:450, w1:1.9986534465450265,w2:1.9986534465450265,bias:-1.714035130013

In [67]:
customModel.predict(X_test_scaled)

2     0.779941
10    0.657769
21    0.216188
11    0.703576
14    0.787099
9     0.826462
0     0.676509
13    0.227108
6     0.337141
dtype: float64

In [68]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 38ms/step


array([[0.747135  ],
       [0.34992838],
       [0.22843245],
       [0.4919505 ],
       [0.7686708 ],
       [0.870502  ],
       [0.4050438 ],
       [0.2609498 ],
       [0.61906004]], dtype=float32)